In [ ]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

URI = "postgresql://postgres@localhost:5433/pdcms"
ENGINE = create_engine('postgresql://postgres@localhost:5433/pdcms')

# Helper classes to execute SQL from files
def parse_sql(filename):
    """SQLAlchemy is not able to handle multiple queries in the same file.
    This function splits the query whenever there is a `;`, which is the SQL delimiter.
    """
    with open(filename, 'r') as f:
        queries = f.read().split(';')
    return([q for q in queries if q.strip() != ''])

def execute_survey_sql(conn, sql_file):
    """Executes each sql query in the file it is given"""
    sql_statements = parse_sql(sql_file)
    for s in sql_statements:
        conn.execute(s)
    return True

## Finding eligible cases and people

The queries that create the population of eligible clients is in two files:

`case_population.sql` selects cases that were closed within the last 6 months, and whose defendant is not on any open cases or was ever 7.30'd.

`people_population.sql` selects people who are eligible to be surveyed. Clients are eligible to be surveyed if they are on one of the cases selected above, are over 18, and speak either English or Spanish.

These queries create two tables in the database called `survey_cases` and `survey_people`. The final query uses both of these interstitial tables.

In [ ]:
with ENGINE.connect() as conn:
    execute_survey_sql(conn, 'case_population.sql')
    execute_survey_sql(conn, 'people_population.sql')

## Creating the final sample frame

`population.sql` puts everything together, and selects the frame from which we will sample.

In [ ]:
with ENGINE.connect() as conn:
    with open('population.sql', 'r') as f:
        FRAME = pd.read_sql(f.read(), conn)

In [ ]:
FRAME.head()

In [ ]:
FRAME.iloc[1]

Some columns have lists in them - change those to strings.

In [ ]:
FRAME['gender'] = FRAME['gender'].apply(lambda x: x[0] if x else None)
FRAME['race'] = FRAME['race'].apply(lambda x: ", ".join(x) if x else None)
FRAME['language'] = FRAME['language'].apply(lambda x: ", ".join(x))
FRAME['client_names'] = FRAME['client_names'].apply(lambda x: ", ".join(x))
FRAME['ethnicity'] = FRAME['ethnicity'].apply(lambda x: 'Hispanic' if 'H' in x else None)

### Further weeding

It's easier to find patterns in text in Python than in Postgresql, so we'll do it here. Below, we'll remove out of county warrants (where the client has a case in another county but is arrested here).

In [ ]:
ny_docket_paterns = (FRAME['docket_number'].str.contains('[0-9]NY*[0-9]', na=False) # docket has NY in it
                     | FRAME['docket_number'].str.contains('CN', na=False) # or CN
                     | FRAME['docket_number'].str.contains('/', na=False) # or is an indictment (which isn't supposed to be there, but whatever)
                     | pd.isnull(FRAME['docket_number'])) # or is blank
FRAME = FRAME.loc[ny_docket_paterns, :]

Keep only cases that have either a case docket or an indictment.

In [ ]:
docket_indictment_blank = (pd.isnull(FRAME.docket_number) & pd.isnull(FRAME.indictment_number))
FRAME = FRAME.loc[~docket_indictment_blank, :]

Indictments are sometimes recorded as dockets, and vice versa. The following moves the docket or indictment number into the correct column, if they were recorded incorrectly.

In [ ]:
indictment_pattern = (FRAME['docket_number'].str.contains('/', na=False) | FRAME['docket_number'].str.contains('-', na=False))
docket_pattern = (FRAME['indictment_number'].str.contains('NY', na=False))

# Copies the indictment number into the indictment column,
# then sets the indictment number in the docket column to None
FRAME.loc[indictment_pattern, 'indictment_number'] = FRAME.loc[indictment_pattern, 'docket_number']
FRAME.loc[indictment_pattern, 'docket_number'] = None

# Copies the docket number into the docket column,
# then sets the docket number in the indictment column to None
FRAME.loc[docket_pattern, 'indictment_number'] = FRAME.loc[docket_pattern, 'indictment_number']
FRAME.loc[docket_pattern, 'indictment_number'] = None

Indictments in PDCMS are recorded irregularly. The following cleans them up a bit.

In [ ]:
import re
indcts = FRAME.loc[~pd.isnull(FRAME.indictment_number), 'indictment_number']

def clean_indictments(indct):
    if indct:
        indct = re.sub('0*([0-9]{4}[A-Z]*)/1', '\\1-201', indct)
        return(re.sub('([0-9]{3}[A-Z]*)/1', '0\\1-201', indct))

FRAME['indictment_number'] = FRAME.indictment_number.apply(lambda x: clean_indictments(x))

Finally, a few lines of housekeeping.

In [ ]:
FRAME.head()

In [ ]:
FRAME[['sw_assigned', 'investigator_assigned']] = FRAME[['sw_assigned', 'investigator_assigned']].fillna(0)

## Who is in the sample frame?

#### Case type count:

Most eligible cases were misdemeanors.

In [ ]:
FRAME.groupby('case_type').agg('size').sort_values()

#### How long did these cases last?

This is a little tricky because of how many people 

In [ ]:
FRAME['age_of_case'] = (FRAME.case_close_date - FRAME.case_open_date).dt.days
FRAME.groupby('case_type').agg({'age_of_case': 'describe'})

#### Client ages:

Age at case open. Unsurprisingly, mostly people in their 20s and 30s.

In [ ]:
# Ages
%matplotlib
(FRAME.case_open_date - FRAME.dob).apply(lambda x: x// np.timedelta64(1, 'Y')).plot.hist(title='Client Ages')

In [ ]:
FRAME.language.value_counts()

In [ ]:
FRAME.gender.value_counts()

In [ ]:
FRAME.describe()

### Add trial information

The following code looks in the trial dataset to see which cases in the sampling frame went to trial.

In [ ]:
trials = pd.read_csv('./data/trials.csv') #clients who went to trial
# sets the column to 0, then sets rows where the indictment or docket number are in the trials dataset to 1
FRAME['went_to_trial'] = 0
FRAME.loc[FRAME.indictment_number.isin(trials.Indictment) | FRAME.docket_number.isin(trials.Docket), 'went_to_trial'] = 1

In [ ]:
FRAME.went_to_trial.value_counts()

In [ ]:
#social_work_referrals = pd.read_csv()
#investigation_referral = pd.read_csv()

In [ ]:
###
# Things that are important to know
FRAME['social_work_referral'] = None
FRAME['investigation_referral'] = None
FRAME['age_group'] = None
FRAME['indicted'] = FRAME['cas_indictment'].apply(
                                lambda x: True if pd.notnull(x) else False)
FRAME['currently_incarcerated'] = None

## Sample!!

Whew.

The following is the actual sampling code, with some quick checking to make sure the sample is roughly similar to the population frame.

In [ ]:
SAMPLE = FRAME.sample(100)

In [ ]:
SAMPLE.describe()

In [ ]:
print(SAMPLE.gender.value_counts())
print(SAMPLE.non_citizen.value_counts())
print(SAMPLE.language.value_counts())
print(SAMPLE.went_to_trial.value_counts())

In [ ]:
SAMPLE = pd.concat([SAMPLE, FRAME[FRAME['language'] == 'SPANISH'].sample(10)])

In [ ]:
SAMPLE.to_csv('./sample.csv', index=False)

In [ ]:
SAMPLE.drop_duplicates(inplace=True)

In [ ]:
SAMPLE.assigned_attorney.value_counts()